In [2]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras import backend as k
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop



Using TensorFlow backend.


In [3]:
data_dir = "/home/siddharth/Documents/ML data/ASL/asl_alphabet_train/asl_alphabet_train"
target_size = (64, 64)
target_dims = (64,64,3) # add channel for RGB
n_classes = 29
val_frac = 0.1
batch_size = 64

data_augmentor = ImageDataGenerator(samplewise_center=True, 
                                    samplewise_std_normalization=True,
                                    rescale=1.0/255.0,
                                    validation_split=val_frac)

train_generator = data_augmentor.flow_from_directory(data_dir, target_size=target_size, batch_size=batch_size, shuffle=True, subset="training")
val_generator = data_augmentor.flow_from_directory(data_dir, target_size=target_size, batch_size=batch_size, subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [7]:
model = Sequential()

model.add(Convolution2D(filters = 16, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (64,64,3)))
model.add(Convolution2D(filters = 16, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Convolution2D(filters = 32, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Convolution2D(filters = 32, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(29, activation = "softmax"))
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [8]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 64, 64, 16)        1216      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 16)        6416      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)       

In [9]:
history=model.fit_generator(train_generator, epochs=5, validation_data=val_generator)

Epoch 1/5
1224/1224 [==============================] - 1245s 1s/step - loss: 1.1177 - accuracy: 0.6562 - val_loss: 0.7256 - val_accuracy: 0.7546
Epoch 2/5
1224/1224 [==============================] - 693s 566ms/step - loss: 0.2746 - accuracy: 0.9105 - val_loss: 0.4936 - val_accuracy: 0.8163
Epoch 3/5
1224/1224 [==============================] - 666s 544ms/step - loss: 0.1835 - accuracy: 0.9427 - val_loss: 0.5261 - val_accuracy: 0.7926
Epoch 4/5
1224/1224 [==============================] - 659s 539ms/step - loss: 0.1558 - accuracy: 0.9535 - val_loss: 0.3360 - val_accuracy: 0.8469
Epoch 5/5
1224/1224 [==============================] - 651s 532ms/step - loss: 0.1468 - accuracy: 0.9574 - val_loss: 1.0703 - val_accuracy: 0.8648


In [10]:
model.save('Sign_model2.h5')